## 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/BOAZ/분석 adv

/content/drive/MyDrive/BOAZ/분석 adv


In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 72.5 MB/s 
     |████████████████████████████████| 453 kB 47.9 MB/s 


In [ ]:
# 데이터 조작 관련
import pandas as pd
import numpy as np
import seaborn as sns
import re
import io

# 한국어 nlp
from konlpy.tag import *   # 모든 형태소분석기 import 하기
#1. hannanum = Hannanum()
#2. kkma = Kkma()
#3. komoran = Komoran()
#4. okt = Okt()

okt = Okt()

### 위 4개중 원하는 형태소분석기를 사용하면 됨 --> okt추천

# 영어 nlp 관련
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image

# 기타
from tqdm import tqdm_notebook, tqdm   # for문 진행상황 눈으로 확인 (loading bar)
import datetime

## top5 불러오기 & 확인

In [ ]:
#top1-5 크롤링 파일 불러오기
df1 = pd.read_excel('2022-08-16_1위 musinsa crawling.xlsx')#,sheet_name = None)
#top1.to_csv("2022-08-16_1위 musinsa crawling.csv")
df2 = pd.read_excel('2022-08-16_2위 musinsa crawling.xlsx')
df3 = pd.read_excel('2022-08-16_3위 musinsa crawling.xlsx')
df4 = pd.read_excel('2022-08-16_4위 musinsa crawling.xlsx')
df5 = pd.read_excel('2022-08-16_5위 musinsa crawling.xlsx')

In [ ]:
df5.head()

,profile,item,content,evaluation
0,"\n여성, 154cm, 45kg\n신고\n",부클 쿠퍼 로고 맨투맨 그린,색이 너무 예뻐서 잘 입고 다니고 있어요 ! 진짜 추천 추천 !!,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
1,"\n남성, 175cm, 60kg\n신고\n",부클 쿠퍼 로고 맨투맨 그린,색이 예뻐서 바로 주문 했어요 ! 다들 꼭 사세요 추천드립니다,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
2,"\n남성, 181cm, 84kg\n신고\n",부클 쿠퍼 로고 맨투맨 그린,티셔츠 넘 이뻐요 청바지나 면바지에도 이뻐요 자주입습니다,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
3,"\n남성, 184cm, 70kg\n신고\n",부클 쿠퍼 로고 맨투맨 네이비,사이즈도 불편하지않고 따뜻해서 봄에 입기 좋은거같아요,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
4,"\n남성, 175cm, 85kg\n신고\n",부클 쿠퍼 로고 맨투맨 그레이,이뻐요 사이즈도 딱 좋고 기모가 있어서 먼지가 좀 나오지만 괜찮아요,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n


In [ ]:
#top1-5 합치기
df = pd.concat([df1,df2,df3,df4,df5])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   profile     3000 non-null   object
 1   item        3000 non-null   object
 2   content     3000 non-null   object
 3   evaluation  3000 non-null   object
dtypes: object(4)
memory usage: 117.2+ KB


## 프로필 수정: df_profile

In [ ]:
# 프로필 성,키, 몸무게로 바꾸기
df_profile = pd.DataFrame(df['profile'].str.split(',').tolist(),columns=['gender','height','weight'])
df_profile.head()

,gender,height,weight
0,\n남성,175cm,68kg\n신고\n
1,\n남성,174cm,68kg\n신고\n
2,\n남성,180cm,76kg\n신고\n
3,\n남성,168cm,66kg\n신고\n
4,\n여성,178cm,58kg\n신고\n


In [ ]:
#df2['gender'] = df2['gender'].str.replace('￦n', '') #regex=True)
df_profile["gender"] = df_profile["gender"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df_profile["weight"] = df_profile["weight"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df_profile["weight"] = df_profile["weight"].str.replace(pat=r'신고', repl=r'', regex=True)
df_profile.head()

,gender,height,weight
0,남성,175cm,68kg
1,남성,174cm,68kg
2,남성,180cm,76kg
3,남성,168cm,66kg
4,여성,178cm,58kg


In [ ]:
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   gender  3000 non-null   object
 1   height  2562 non-null   object
 2   weight  2560 non-null   object
dtypes: object(3)
memory usage: 70.4+ KB


## 평가 수정: df_evaluation

In [ ]:
df_evaluation = pd.DataFrame(df['evaluation'].str.split(pat=r'[^\w]').tolist(),columns=['size','brightness','color','thickness','1','2','3','4','5','7','6','8'])
df_evaluation.head()

,size,brightness,color,thickness,1,2,3,4,5,7,6,8
0,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,얇아요,,
1,,,사이즈,커요,밝기,보통이에요,색감,보통이에요,두께감,얇아요,,
2,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,얇아요,,
3,,,사이즈,보통이에요,밝기,보통이에요,색감,보통이에요,두께감,보통이에요,,
4,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,보통이에요,,


In [ ]:
df_evaluation.columns =['1','2','3','size','4','brightness','5','color','6','thickness','7','8']
df_evaluation.head()

,1,2,3,size,4,brightness,5,color,6,thickness,7,8
0,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,얇아요,,
1,,,사이즈,커요,밝기,보통이에요,색감,보통이에요,두께감,얇아요,,
2,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,얇아요,,
3,,,사이즈,보통이에요,밝기,보통이에요,색감,보통이에요,두께감,보통이에요,,
4,,,사이즈,보통이에요,밝기,밝아요,색감,선명해요,두께감,보통이에요,,


In [ ]:
df_evaluation.drop(columns=['1','2','3','4','5','6','7','8'],inplace=True)
df_evaluation.head()

,size,brightness,color,thickness
0,보통이에요,밝아요,선명해요,얇아요
1,커요,보통이에요,보통이에요,얇아요
2,보통이에요,밝아요,선명해요,얇아요
3,보통이에요,보통이에요,보통이에요,보통이에요
4,보통이에요,밝아요,선명해요,보통이에요


In [ ]:
df_evaluation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   size        3000 non-null   object
 1   brightness  3000 non-null   object
 2   color       3000 non-null   object
 3   thickness   3000 non-null   object
dtypes: object(4)
memory usage: 93.9+ KB


## 최종 df 도출

In [ ]:
df = df.reset_index()
df.drop(columns=['profile','evaluation'],inplace=True)
df_final = pd.concat([df_profile,df,df_evaluation], axis = 1)
df_final.head()


,gender,height,weight,index,item,content,size,brightness,color,thickness
0,남성,175cm,68kg,0,에센셜 쿨 코튼 2-PACK 티셔츠,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티",보통이에요,밝아요,선명해요,얇아요
1,남성,174cm,68kg,1,에센셜 쿨 코튼 2-PACK 티셔츠,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당,커요,보통이에요,보통이에요,얇아요
2,남성,180cm,76kg,2,에센셜 쿨 코튼 2-PACK 티셔츠,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요,보통이에요,밝아요,선명해요,얇아요
3,남성,168cm,66kg,3,에센셜 쿨 코튼 2-PACK 티셔츠,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요,보통이에요,보통이에요,보통이에요,보통이에요
4,여성,178cm,58kg,4,에센셜 쿨 코튼 2-PACK 티셔츠,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~,보통이에요,밝아요,선명해요,보통이에요


In [ ]:
df_final.drop(columns=['index'],inplace=True)
df_final.head()

,gender,height,weight,item,content,size,brightness,color,thickness
0,남성,175cm,68kg,에센셜 쿨 코튼 2-PACK 티셔츠,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티",보통이에요,밝아요,선명해요,얇아요
1,남성,174cm,68kg,에센셜 쿨 코튼 2-PACK 티셔츠,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당,커요,보통이에요,보통이에요,얇아요
2,남성,180cm,76kg,에센셜 쿨 코튼 2-PACK 티셔츠,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요,보통이에요,밝아요,선명해요,얇아요
3,남성,168cm,66kg,에센셜 쿨 코튼 2-PACK 티셔츠,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요,보통이에요,보통이에요,보통이에요,보통이에요
4,여성,178cm,58kg,에센셜 쿨 코튼 2-PACK 티셔츠,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~,보통이에요,밝아요,선명해요,보통이에요


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gender      3000 non-null   object
 1   height      2562 non-null   object
 2   weight      2560 non-null   object
 3   item        3000 non-null   object
 4   content     3000 non-null   object
 5   size        3000 non-null   object
 6   brightness  3000 non-null   object
 7   color       3000 non-null   object
 8   thickness   3000 non-null   object
dtypes: object(9)
memory usage: 211.1+ KB


In [ ]:
#csv파일로 저장
df_final.to_csv(
    path_or_buf = '/content/drive/MyDrive/BOAZ/분석 adv/covernat_top5.csv',
    index = False,       # 0부터 시작하는 자연수 인덱스는 의미가 없음.
    # columns = ['col1, 'col2', 'col3'],
    sep = ',',      # 구분기호 설정
    encoding = 'utf-8-sig',
    #float_format='%f',     # 숫자 자리수 모두 표시
    na_rep = '',      # 결측값을 ''으로 채우기
)     # 해당 파일이 열려있으면 안됨.